In [2]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/home/javiermunoz/Universidad/MasterBDyDS/TFM/TFM_DSBD'

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import keras
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Dropout, Flatten

import os 
from src.config import LM_PER_VIDEO, BASE_OUTPUT, TOTAL_LANDMARKS
from sklearn.utils import shuffle
from utils import load_dict

/home/javiermunoz/anaconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [4]:
X_tens = load_dict("./data/subset_10_lsa_64/pickl_files/npy_db_x.pkl")
Y_enc = load_dict("./data/subset_10_lsa_64/pickl_files/npy_db_y.pkl")
le_mapping = load_dict("./data/subset_10_lsa_64/pickl_files/labels_map.pkl")

In [5]:
# Assign the dataset splits
X_train, X_val, X_test = X_tens['train'], X_tens['val'], X_tens['test']
y_train, y_val, y_test = Y_enc['train'], Y_enc['val'], Y_enc['test']
n_classes = len(le_mapping)

# shuffling the data
X_train, y_train = shuffle(X_train, y_train)
X_val, y_val = shuffle(X_val, y_val)
X_test, y_test = shuffle(X_test, y_test)

In [6]:
# Mediapipe normalizes data for us
X_train[0].all() < 1 

True

In [7]:
X_train.shape, X_val.shape, X_test.shape

((394, 20, 1662), (60, 20, 1662), (46, 20, 1662))

In [8]:
y_train.shape, y_val.shape, y_test.shape

((394,), (60,), (46,))

In [9]:
model = Sequential([
    LSTM(units=128, activation='relu', return_sequences=True, input_shape=(LM_PER_VIDEO, TOTAL_LANDMARKS)),
    Dropout(0.2),
    LSTM(units=256, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(units=128, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(units=64, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dense(units=n_classes, activation='softmax'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 128)           916992    
                                                                 
 dropout (Dropout)           (None, 20, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 20, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 20, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 20, 128)           197120    
                                                                 
 dropout_2 (Dropout)         (None, 20, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                4

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
# CReate a customized callback so when val_acc > 94% it stops training
from keras.callbacks import EarlyStopping, ModelCheckpoint

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

In [12]:
epochs = 500
bst_model_path = BASE_OUTPUT + "best_weights.h5"

callbacks = [
    MyThresholdCallback(threshold=0.94),
    #ModelCheckpoint(filepath = bst_model_path, save_best_only=True, save_weights_only=True, monitor="accuracy")
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    callbacks=callbacks,
    batch_size=32
)

Epoch 1/500
13/13 [==============================] - 8s 270ms/step - loss: 2.2988 - accuracy: 0.1015 - val_loss: 2.3079 - val_accuracy: 0.0333
Epoch 2/500
13/13 [==============================] - 3s 261ms/step - loss: 2.2886 - accuracy: 0.1015 - val_loss: 2.2896 - val_accuracy: 0.0333
Epoch 3/500
13/13 [==============================] - 4s 293ms/step - loss: 2.2434 - accuracy: 0.1523 - val_loss: 2.2063 - val_accuracy: 0.1500
Epoch 4/500
13/13 [==============================] - 3s 245ms/step - loss: 2.2072 - accuracy: 0.1802 - val_loss: 2.1635 - val_accuracy: 0.2833
Epoch 5/500
13/13 [==============================] - 3s 264ms/step - loss: 2.1050 - accuracy: 0.2081 - val_loss: 2.0189 - val_accuracy: 0.2667
Epoch 6/500
13/13 [==============================] - 3s 270ms/step - loss: 1.9436 - accuracy: 0.2487 - val_loss: 1.8083 - val_accuracy: 0.3167
Epoch 7/500
13/13 [==============================] - 3s 228ms/step - loss: 1.9361 - accuracy: 0.2843 - val_loss: 1.9277 - val_accuracy: 0.3500

In [13]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
2/2 [==============================] - 0s 24ms/step - loss: 0.5199 - accuracy: 0.8478
test loss, test acc: [0.5198689699172974, 0.8478260636329651]
